<a href="https://colab.research.google.com/github/AlGhifaryAN/MachineLearningTasksTK44G4/blob/main/UTS_ML_XGBoost_Google_Play_Storen_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Nama    : Al Ghifary Akmal Nasheeri
#NIM     : 1103201242
#Kelas   : TK-44-G4
#Model   : XGBoost
#Dataset : Google Play Store Apps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries dan Load Dataset

In [ ]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Mendefinisikan path dataset Google Play Store Apps CSV
playstore_dataset_path = '/content/drive/MyDrive/Dataset/googleplaystore.csv'

# Mendefinisikan path dataset Google Play Store Apps CSV
playstore_data = pd.read_csv(playstore_dataset_path)
print(playstore_data.head())

                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend 

Data Preprocessing

In [ ]:
# Mengubah kolom 'Price' menjadi numerik, mengganti nilai non-numerik dengan 0
playstore_data['Price'] = playstore_data['Price'].apply(lambda x: 0 if x == '0' else float(x[1:]) if '$' in x else 0)
playstore_data['Size'] = playstore_data['Size'].replace('Varies with device', '0')
playstore_data['Size'] = playstore_data['Size'].apply(lambda x: str(x).replace('M', '') if 'M' in str(x) else str(x))
playstore_data['Size'] = playstore_data['Size'].apply(lambda x: str(x).replace('k', '') if 'k' in str(x) else str(x))
playstore_data['Size'] = pd.to_numeric(playstore_data['Size'].replace(',', '', regex=True), errors='coerce').fillna(0)
playstore_data['Installs'] = playstore_data['Installs'].apply(lambda x: 0 if not x.isdigit() else int(x[:-1].replace(',', '')) if '+' in x else 0)
playstore_data['Reviews'] = pd.to_numeric(playstore_data['Reviews'], errors='coerce').fillna(0)

# Melakukan encoding kolom 'Content Rating' dan memilih fitur-fitur
playstore_data = pd.get_dummies(playstore_data, columns=['Content Rating'], drop_first=True)
cols_to_use_playstore = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']
playstore_data = playstore_data[cols_to_use_playstore].dropna()
X_playstore = playstore_data.drop('Rating', axis=1)
y_playstore = playstore_data['Rating']

Menampilkan Data Frame yang Digunakan

In [ ]:
# Menampilkan frame yang digunakan untuk XGBoost
print("Frame yang digunakan untuk XGBoost:")
print(X_playstore.head())


Frame yang digunakan untuk XGBoost:
    Reviews  Size  Installs  Price
0     159.0  19.0         0    0.0
1     967.0  14.0         0    0.0
2   87510.0   8.7         0    0.0
3  215644.0  25.0         0    0.0
4     967.0   2.8         0    0.0


Split Data dan Penyetelan Hyperparameter

In [ ]:
# Memisahkan data ke dalam set pelatihan dan validasi
X_train_playstore, X_valid_playstore, y_train_playstore, y_valid_playstore = train_test_split(X_playstore, y_playstore, test_size=0.2, random_state=42)

# Penyetelan hyperparameter menggunakan GridSearchCV
model_grid = XGBRegressor()
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300]
}
grid_search = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_playstore, y_train_playstore)
best_n_estimators = grid_search.best_params_['n_estimators']
print("Best n_estimators:", best_n_estimators)


Best n_estimators: 100


Pelatihan dan Penyesuaian Model dengan n_estimators Berbeda

In [ ]:
# Menginisialisasi my_model dengan XGBRegressor dan melatih dengan seluruh data
my_model = XGBRegressor()
my_model.fit(X_playstore, y_playstore)

# Menyesuaikan model dengan n_estimators yang berbeda dan menyimpan hasil MAE
n_estimators_list = [100, 150, 200, 250, 300]
mae_results = []

for n_estimators in n_estimators_list:
    model_grid = XGBRegressor(n_estimators=n_estimators)
    model_grid.fit(X_train_playstore, y_train_playstore)

    predictions = model_grid.predict(X_valid_playstore)
    mae = mean_absolute_error(predictions, y_valid_playstore)

    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")
    mae_results.append(mae)

n_estimators: 100, Mean Absolute Error: 0.35572372828883575
n_estimators: 150, Mean Absolute Error: 0.35796208865105666
n_estimators: 200, Mean Absolute Error: 0.3628964827307515
n_estimators: 250, Mean Absolute Error: 0.36379132146133053
n_estimators: 300, Mean Absolute Error: 0.36743499448994055
